# Задание 1

Часть 1. Работа с данными

Входные данные для тестового задания можно найти в таблице

**Ваша задача - подготовить и обработать исходные данных так, чтобы их можно было использовать во второй части задания**

**Требования к выходным данным:**
     
     1) В выходной таблице должны остаться только следующие колонки:
     area, cluster, cluster_name, keyword, x, y, count, color, где
     area - область,
     cluster - номер кластера,
     cluster_name - название кластера,
     keyword - словосочетание,
     count - показатель,
     x и y - координаты для диаграммы рассеяния,
     color - цвет точки на карте для данного словосочетания

    2) Колонку color нужно добавить самостоятельно - цвета вы можете взять из цветовых палеток Tableu или по своему усмотрению.
    
    3) Цвет задается каждому словосочетанию согласно следующими правилам:
    - внутри одной области цвета словосочетаний в одном кластере должны быть одинаковые, в разных - отличаться (например, у "Кластер 1" все слова будут окрашены в красный, у "Кластер 2" - в зеленый и т.д.)
    - цвета кластеров в разных областях могут повторяться
    - цвета кластеров в разных областях с разным номером не имеют никакой связи (у одной области [area] слова из "Кластер 1" могут быть красного цвета, в другой области у слов из "Кластер 1" может быть другой цвет)
        
        4) Не должно быть дубликатов слов в одной и той же области (area), но словосочетание может повторяться из area в area
        
        5) Колонки должны называться именно так, как указано в п.1
        
        6) Сортировка должна происходить по колонкам area, cluster, cluster_name, count (по count значения сортируются в убывающем порядке, в остальных - по возрастающему).
        
        7) Количество переданных в исходных ключевых слов должно совпадать с количество слов в выходных данных (за исключением дублированных строк или строк с пустыми\неформатными значениями по ключевым показателям [перечислены в п. 1], если такие имеются).
        
        8) Никакие другие особенности оформления не должны учитываться при обработке данных (заливка и пр.)
        
        9) Выходные данные должны быть аккуратно оформлены (заголовки закреплены, включен фильтр)
    

Формат представления выходных данных: google spreadsheet-таблица.

Выполнение данной работы желательно с помощью библиотеки pandas (Python)


Импортируем необходимые библиотеки для работы

In [ ]:
!pip install seaborn

In [ ]:
pip install adjustText

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats as st
import matplotlib.patches as mpatches
from adjustText import adjust_text

Выгрузим данные

In [ ]:
data = pd.read_csv('C:\\Users\\STAS\\Downloads\\tz_data.csv')
data.info()

8 колонок. Типы данных - object и float64

Нам нужны только колонки area, cluster, cluster_name, keyword, x, y, count, color

Удалим колонку good (1) и выведем первые 20 строк таблицы

In [ ]:
data = data.drop('good (1)', axis=1)

In [ ]:
display(data.head(20))

Отлично - теперь у нас нет столбца good (1).

Дальше нам нужно:

- перевести count и cluster в int, y - в float

- добавить столбец color

Чтобы приступить к дальнейшей работе с данными, нужно проверить данные на предмет дубликатов и пропусков, при наличии которых избавиться от них

Проверим данные на предмет дубликатов и удалим их

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

Дубликатов не было 

Проверим на предмет пропусков

In [ ]:
display(data.isnull().sum())

Пропусков мало, их можно удалить

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

Мы удалили пропуски.
Теперь удалим нечисловые значения из столбцов, которые хотим перевести в int

Создадим список этих столбцов 

In [ ]:
columns_to_convert = ['count', 'y', 'cluster']

Теперь проверим наличие нечисловых значений в каждом из них

In [ ]:
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

Применим эту функцию к каждому столбцу, чтобы получить строки с нечисловыми значениями

In [ ]:
for col in columns_to_convert:
    non_numeric_rows = data[~data[col].apply(is_numeric)]
    if len(non_numeric_rows) > 0:
        print(f"Column '{col}' contains non-numeric values:")
        print(non_numeric_rows)

Удалим значения

In [ ]:
data = data[data['count'].apply(is_numeric)]

In [ ]:
data = data[data['y'].apply(is_numeric)]

Теперь преобразуем столбцы в int и float

In [ ]:
data['count'] = data['count'].astype(int)
data['cluster'] = data['cluster'].astype(int)
data['y'] = data['y'].astype(float)

In [ ]:
data.info()

Нужно добавить столбец color. Добавляем колонку color. Используем Tableau color palette

In [ ]:
colors = ['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF']

Теперь сделаем словарь цветов для каждой комбинации area-cluster

In [ ]:
data = data.dropna(subset=['area', 'cluster'])

color_map = {}
for i, (area, cluster) in enumerate(data[['area', 'cluster']].drop_duplicates().values):
    color_map[(area, cluster)] = colors[i % len(colors)]

data['color'] = data.apply(lambda row: color_map[(row['area'], row['cluster'])], axis=1)

In [ ]:
data.head(20)

In [ ]:
data.info()

Отлично! Отсортируем данные

In [ ]:
data = data.sort_values(by=['area', 'cluster', 'cluster_name', 'count'], ascending=[True, True, True, False])

И сохраним обработанные данные в новый csv файл

In [ ]:
data.to_csv('processed_data.csv', index=False)

Файл с обработанными данными:

https://docs.google.com/spreadsheets/d/1jIhDlKLf6azqm9kZVHKz9EFR0umR5o_NJ9QmihqATZo/edit?usp=sharing

# Задание 2

Часть 2. Построение графиков

На основании обработанных данных постройте по одной диаграмме рассеяния для каждой области (area) (пример внешнего вида см. в приложенном svg-файле).

**Строгие требования к визуализации:**

    - Наличие Footer-подписи на изображении.
    - Наличие легенды цветов и кластеров.
    - Минимизация наложения (слепливания) подписей к друг на друга (постарайтесь сделать так, чтобы наложение было минимальным).

**Желательные требования к визуализации:**

    - Перенос слишком длинных словосочетаний (например, слова длиннее 15 символов, можно разбить на "solar\n cell").
    - Обводка точек.

**Формат представления выходных данных:**

Png-файлы размером не менее 1500х1500 пикселей с визуализациями для каждой области (area).

**Выполнение данной работы желательно с помощью одной из библиотек:**

Matploptlib  (Python)
plotly (Python) и т.п.

**Строгие требования к результатам:**

- Код для первой части задания (с комментариями и приложенным README)

- Код для второй части задания (с комментариями и приложенным README)

- Таблица с трансформированными данными (открыть доступ по ссылке)

 - Набор визуализаций (выложить на облачное хранилище)


**Формат представления кода:**

GitHub

Визуализаций и таблиц - Google Docs


Выгрузим новую таблицу

In [ ]:
df = pd.read_csv('C:\\Users\\STAS\\processed_data.csv')
df.info()

Построим диаграмму рассеяния для каждой области

In [ ]:
def plot_scatter_for_area(area_name, data):
    fig, ax = plt.subplots(figsize=(15, 15))

    area_data = df[df['area'] == area_name]
    clusters = area_data['cluster'].unique()
    texts = []

    for cluster in clusters:
        cluster_data = area_data[area_data['cluster'] == cluster]
        ax.scatter(cluster_data['x'], cluster_data['y'], color=cluster_data['color'].iloc[0], edgecolor='black', label=f"Кластер {cluster}", s=100)
        
        for i, row in cluster_data.iterrows():
            label = "\n".join(row['keyword'].split(" "))
            texts.append(ax.text(row['x'], row['y'], label))

    adjust_text(texts)
            
    ax.set_title(f"Диаграмма рассеяния для области {area_name}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend(title="Кластеры", loc="upper right")
    
    ax.annotate("Выполнено Панковой Славой", (0.5, -0.08), xycoords="axes fraction", ha="center", fontsize=10)

    plt.tight_layout()
    area_filename = area_name.replace('\\', '-')
    plt.savefig(f"{area_filename}.png", dpi=150)

    plt.show()

areas = df['area'].unique()
for area in areas:
    plot_scatter_for_area(area, data)


Файл с визуализацией:

https://docs.google.com/document/d/18trHKXHk9jbaLQONTaaZMXuI-vN3xHtMdmRR_rutRaA/edit?usp=sharing